In [1]:
import torch
from combine_models import CombineGeneral
import config_model
import os
import pickle
from torch_geometric.data import DataLoader
from train import train_model
from config import DEVICE

/home/michpir/Documents/PROJECTS/NeuralSpecLibrary/venv/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))
/home/michpir/Documents/PROJECTS/NeuralSpecLibrary/venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


In [2]:
configuration = config_model.transformer_cnn_model

In [3]:
model_body = configuration["embedding"]
model_head = configuration["head"]

In [4]:
save_path = "/home/michpir/Documents/PROJECTS/output"

In [5]:
checkpoint = torch.load(os.path.join(save_path, f"model_epoch_3.pt"))
model_body.load_state_dict(checkpoint['model_body_state_dict'])
model_head.load_state_dict(checkpoint['model_head_state_dict'])

<All keys matched successfully>

In [6]:
combine_general_model = configuration["combine"](model_body, model_head, configuration["mass_shift"])

# Continue training

In [7]:
# Set path to datasets
train_dataset_path = configuration["training"]["train_dataset_path"]
validation_dataset_path = configuration["training"]["validation_dataset_path"]
test_validation_path = configuration["training"]["test_dataset_path"]


with open(validation_dataset_path, 'rb') as handle:
    validation_dataset = pickle.load(handle)
with open(train_dataset_path, 'rb') as handle:
    train_dataset = pickle.load(handle)
with open(test_validation_path, 'rb') as handle:
    test_dataset = pickle.load(handle)

    # Pick first 5000 from test dataset during training, to see validation
    # test_dataset = test_dataset[:5000]

from torch.utils.data import Subset
import random
def create_subset_indices(dataset, num_instances):
    return random.sample(range(len(dataset)), min(len(dataset), num_instances))

    # Create subset indices for each dataset
train_indices = create_subset_indices(train_dataset, 5000)
validation_indices = create_subset_indices(validation_dataset, 5000)
test_indices = create_subset_indices(test_dataset, 5000)

    # Create subsets from the datasets using the generated indices
train_subset = Subset(train_dataset, train_indices)
validation_subset = Subset(validation_dataset, validation_indices)
test_subset = Subset(test_dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=configuration["training"]["batch_size"], shuffle=True)
validation_loader = DataLoader(validation_subset, batch_size=configuration["training"]["batch_size"], shuffle=True)
test_loader = DataLoader(test_subset, batch_size=configuration["training"]["batch_size"],
                                   shuffle=True)

# Initialize loss function
loss_fn = configuration["training"]["loss_fun"]

# Initialize optimizer (and scheduler if needed)
learning_rate = configuration["training"]["learning_rate"]/0.5
optimizer = configuration["training"]["optimizer"](combine_general_model.parameters(), lr=learning_rate)

# Assume scheduler is optional, hence the use of get to return None if not found
step_size = configuration["training"]["step_size"]
gamma = configuration["training"]["gamma"]
scheduler = configuration["training"]["scheduler"](optimizer, step_size, gamma)

    # Training loop
train_model(
    model=combine_general_model,
    model_config=configuration,
    train_loader=train_loader,
    loss_fn=loss_fn,
    optimizer=optimizer,
    epochs=configuration["training"]["epochs"],
    device=DEVICE,
    val_loader=validation_loader,
    test_loader=test_loader,
    save_every=configuration["training"]["save_every"],
    report_every=configuration["training"]["report_every"],
    save_path=os.path.join(configuration["training"]["save_path"], "fine_tune"),
    scheduler=scheduler
    )

/home/michpir/Documents/PROJECTS/NeuralSpecLibrary/venv/lib/python3.10/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/home/michpir/Documents/PROJECTS/NeuralSpecLibrary/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/home/michpir/Documents/PROJECTS/NeuralSpecLibrary/heads/heads_utils.py:63: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.spar

KeyboardInterrupt: 

# 

In [10]:
optimizer = configuration["training"]["optimizer"](combine_general_model.parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])